# Deploying and invoking Drift Detector Dataflow Flex template

In [ ]:
%cd ../drift_detector

In [ ]:
import json
from datetime import datetime
import time

## Build the Flex template docker image

In [ ]:
template_name = 'drift-detector'

project_id = !(gcloud config get-value core/project)
project_id = project_id[0]
template_image = 'gcr.io/{}/{}:latest'.format(project_id, template_name)
location = 'us-central1'

In [ ]:
!gcloud builds submit --tag {template_image} .

## Deploy the Flex template using `gcloud`

In [ ]:
flex_templates_location = 'gs://mlops-dev-workspace/flex-templates'
template_path = '{}/{}.json'.format(flex_templates_location, template_name)
metadata_file = 'metadata.json'

In [ ]:
!gcloud beta dataflow flex-template build {template_path} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file}

## Trigger a run of the template using REST API

In [ ]:
import googleapiclient.discovery

service = googleapiclient.discovery.build('dataflow', 'v1b3')

parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/2020-05-19T03:50-2020-05-19T03:52',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

request = service.projects().locations().flexTemplates().launch(
    location=location,
    projectId=project_id,
    body=body)

response = request.execute()
response

## Trigger a run of the template using REST API - Curl

In [ ]:
service_url = 'https://dataflow.googleapis.com/v1b3/projects/{}/locations/us-central1/flexTemplates:launch'.format(project_id)
headers_content = 'Content-Type: application/json'
access_token = !(gcloud auth print-access-token) 
headers_auth = 'Authorization: Bearer {}'.format(access_token[0])
parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/2020-05-19T03:50-2020-05-19T03:52',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

json_body = json.dumps(body)

In [ ]:
!curl -X POST \
  "{service_url}" \
  -H "{headers_content}" \
  -H "{headers_auth}" \
  -d '{json_body}'

## Trigger a run of the template using `gcloud`

In [ ]:
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
g_parameters = ','.join(['{}={}'.format(key,value) for key, value in parameters.items()])

!gcloud beta dataflow flex-template run {job_name} \
--template-file-gcs-location {template_path} \
--parameters {g_parameters}